In [1]:
import os,sys
import angr


filePath = '/home/nahid/reverse/binaries/gnuit/src/gitfm'
proj = angr.Project(filePath)

obj = proj.loader.main_object


fh = open(filePath, 'rb')
bin_bytearray = bytearray(fh.read())


import os,sys
import angr
from capstone import *

from elftools.elf.elffile import ELFFile






WARNING  | 2023-04-30 00:24:19,579 | cle.loader     | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [2]:
code_section = obj.find_section_containing(obj.entry)

with open(filePath, 'rb') as f:
    elffile = ELFFile(f)

    entry_point = elffile._parse_elf_header().e_entry
    segment_load_address = elffile.get_segment(0).header.p_vaddr
    entry_offset = entry_point - segment_load_address


        
print(entry_point , segment_load_address,entry_offset)

20208 64 20144


In [3]:
def int_list_to_hex_list(int_list):
    hex_list =[]
    for i in int_list:
        hex_list.append(hex(i))
    return hex_list


def hex_list_to_int_list(hex_list):
    int_list =[]
    for h in hex_list:
        int_list.append(int(i,0))
    return int_list

In [4]:

from collections import defaultdict
import posixpath
from elftools.elf.elffile import ELFFile

lineprogram_addresses = []

with open(filePath, 'rb') as f:
    elffile = ELFFile(f)
    
    
    print(dir(elffile))
    print('\n\n\n',elffile.header,'\n\n')
    

    text_section = elffile.get_section_by_name(".text")
#     bin_bytearray = text_section.data()
    textSectionLen = len(bin_bytearray)
    textSectionStart = text_section.header.sh_addr
    textSectionEnd = textSectionStart + textSectionLen
    program_entry_point = elffile._parse_elf_header().e_entry
    
    print('---------',text_section)
    print(dir(text_section))
    
    print(text_section.header)
    
    
    
    
    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
        

    dwarfinfo = elffile.get_dwarf_info()
    

    cc = 0
    for CU in dwarfinfo.iter_CUs():
        print('  Found a compile unit at offset %s, length %s' % (
            CU.cu_offset, CU['unit_length']))

        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue




        for le in line_program.get_entries():
            if le.state is not None:
                
                lineprogram_addresses.append(le.state.address)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_decompress_dwarf_section', '_get_section_header', '_get_section_header_stringtable', '_get_section_name', '_get_segment_header', '_identify_file', '_make_elf_hash_section', '_make_gnu_hash_section', '_make_gnu_verdef_section', '_make_gnu_verneed_section', '_make_gnu_versym_section', '_make_section', '_make_section_name_map', '_make_segment', '_make_sunwsyminfo_table_section', '_make_symbol_table_index_section', '_make_symbol_table_section', '_parse_elf_header', '_read_dwarf_section', '_section_header_stringtable', '_section_name_map', '_section_offset', '_segment_offset', 'address_offsets', 'close', 'e_ide

In [5]:
print( len(lineprogram_addresses))
print(min(lineprogram_addresses), max(lineprogram_addresses))

13616
20192 155678


In [6]:
JUMP_OPCODES = ['JO','JNO','JS','JNS','JE','JZ','JNE','JNZ','JB','JNAE','JC','JNB','JAE','JNC','JBE','JNA','JA',
'JNBE','JL','JNGE','JGE','JNL','JLE','JNG','JG','JNLE','JP','JPE','JNP','JPO','JCXZ','JECXZ']


#took inspiration from capstone github sample code 

from capstone import *
from pprint import pprint


# textSectionStart = code_section.offset 
# textSectionLen =  code_section.filesize

textSectionEnd = textSectionStart+textSectionLen

print('Start: ',hex(code_section.offset) , "end: ", hex(textSectionEnd))

md = Cs(CS_ARCH_X86, CS_MODE_64)


visited_addresses = []
entry_addresses = []
address_and_instructions = []

entry_addresses.append(program_entry_point)
entry_addresses += lineprogram_addresses
counter = 0


call_addresses = []
while len(entry_addresses)>0 :
    
    
    print('ENTRIES REMAINING',len(entry_addresses))
    current_entry_address = entry_addresses.pop(0)
    
    if current_entry_address in visited_addresses:
        continue
    

    CODE = bin_bytearray[current_entry_address: textSectionEnd]
    if len(CODE)==0:
        continue
    for i in md.disasm(CODE, current_entry_address):

        print("0x%x:\t%s\t%s \t%s" %(i.address, i.mnemonic, i.op_str ,i.size))  
        

        address_and_instructions.append([i.address, i.mnemonic, i.op_str])
        
        visited_addresses.append(i.address)

        # IF there is a conditional jump
        if i.mnemonic.upper() in JUMP_OPCODES:
            try:
                if int(i.op_str,0)>=0:
                    if i.op_str not in visited_addresses:
                        print("\t\t\t> %s" %( i.op_str  ))  
                        
                        print('->',hex(i.address + i.size),(i.address + i.size),(i.op_str,0) )
                        entry_addresses.append(int(i.address + i.size))
                        entry_addresses.append(int(i.op_str,0))
                break
            except: 
                print('\t\t->->->                               indirect BRUNCH')
                break
        
        #direct jump  TODO: check all versions
        if i.mnemonic.upper() in ['JMP']: #TODO handle JMP 0xACDC:0x5578
            print(i.op_str)
            try:
                if int(i.op_str,0)>=0:
                    
                    if int(i.op_str,0)>=textSectionStart and int(i.op_str,0)<=textSectionEnd:
                        entry_addresses.append(int(i.op_str,0))
                break
                    
                    
            except:
                print('\t\t->->->                                  indirect JMP')
                break
        if i.mnemonic.upper() in ['RET', 'LEAVE']:
#             end of basic block
            break
#         TODO: check all versions
        if i.mnemonic.upper() in ["CALL"]:
            try:
                if int(i.op_str,0) >= 0:
                    call_addr = int(i.op_str,0)
                    print(call_addr, textSectionStart,textSectionEnd )
                    if call_addr <textSectionStart or call_addr>textSectionEnd:
                        print("EXIT"*100)
                    else:
                        print("YYYY"*100)
                        call_addresses.append(call_addr)
            except:
                
                break
                

Start:  0x4ee0 end:  0x6dcc8
ENTRIES REMAINING 13617
0x4ef0:	endbr64	 	4
0x4ef4:	xor	ebp, ebp 	2
0x4ef6:	mov	r9, rdx 	3
0x4ef9:	pop	rsi 	1
0x4efa:	mov	rdx, rsp 	3
0x4efd:	and	rsp, 0xfffffffffffffff0 	4
0x4f01:	push	rax 	1
0x4f02:	push	rsp 	1
0x4f03:	xor	r8d, r8d 	3
0x4f06:	xor	ecx, ecx 	2
0x4f08:	lea	rdi, [rip + 0x28fb] 	7
0x4f0f:	call	qword ptr [rip + 0x2a0a3] 	6
ENTRIES REMAINING 13616
0x4fd9:	endbr64	 	4
0x4fdd:	push	rbp 	1
0x4fde:	mov	rbp, rsp 	3
0x4fe1:	mov	eax, dword ptr [rip + 0x2b9f5] 	6
0x4fe7:	cmp	eax, 6 	3
0x4fea:	jle	0x501a 	2
			> 0x501a
-> 0x4fec 20460 ('0x501a', 0)
ENTRIES REMAINING 13617
ENTRIES REMAINING 13616
ENTRIES REMAINING 13615
0x4fec:	mov	eax, dword ptr [rip + 0x2a036] 	6
0x4ff2:	test	eax, eax 	2
0x4ff4:	je	0x5008 	2
			> 0x5008
-> 0x4ff6 20470 ('0x5008', 0)
ENTRIES REMAINING 13616
ENTRIES REMAINING 13615
0x4ff6:	mov	eax, dword ptr [rip + 0x2b9e4] 	6
0x4ffc:	cmp	eax, 0xb 	3
0x4fff:	jle	0x501a 	2
			> 0x501a
-> 0x5001 20481 ('0x501a', 0)
ENTRIES REMAINING 13616
E

0x5164:	mov	eax, dword ptr [rbp - 0x14] 	3
0x5167:	movsxd	rcx, eax 	3
0x516a:	mov	eax, dword ptr [rip + 0x2b86c] 	6
0x5170:	sub	eax, 3 	3
0x5173:	movsxd	rdx, eax 	3
0x5176:	mov	rax, qword ptr [rip + 0x2b793] 	7
0x517d:	mov	r8, rcx 	3
0x5180:	mov	rcx, rdx 	3
0x5183:	mov	edx, 1 	5
0x5188:	mov	esi, 0 	5
0x518d:	mov	rdi, rax 	3
0x5190:	call	0x11348 	5
70472 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
0x5195:	mov	eax, dword ptr [rbp - 0x18] 	3
0x5198:	movsxd	rcx, eax 	3
0x519b:	mov	eax, dword ptr [rip + 0x2b83b] 	6
0x51a1:	sub	eax, 3 	3
0x51a4:	movsxd	rdx, eax 	3
0x51a7:	mov	eax, dword ptr [rbp - 0x14] 	3
0x51aa:	movsxd	rsi, ea

0x5486:	mov	rax, qword ptr [rip + 0x2b483] 	7
0x548d:	mov	rdi, rax 	3
0x5490:	call	0x1504f 	5
86095 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
0x5495:	mov	rdi, rax 	3
0x5498:	call	0xd875 	5
55413 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
0x549d:	jmp	0x54a0 

0x5733:	mov	eax, 0 	5
0x5738:	call	0xe569 	5
58729 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
0x573d:	jmp	0x598f 	5
0x598f
ENTRIES REMAINING 13521
ENTRIES REMAINING 13520
0x5742:	mov	eax, 0 	5
0x5747:	call	0xe655 	5
58965 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

0x5874:	mov	eax, dword ptr [rbp - 8] 	3
0x5877:	and	eax, 1 	3
0x587a:	test	eax, eax 	2
0x587c:	je	0x5982 	6
			> 0x5982
-> 0x5882 22658 ('0x5982', 0)
ENTRIES REMAINING 13509
ENTRIES REMAINING 13508
0x5882:	mov	eax, 0 	5
0x5887:	call	0xf009 	5
61449 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
0x588c:	jmp	0x5982 	5
0x5982
ENTRIES REMAINING 13508
ENTRIES REMAINING 13507
0x5891:	mov	eax, dword ptr [rbp - 8] 	3
0x5894:	and	eax, 1 	3
0x5897:	test	eax, eax 	2
0x5899:	je	0x5985 	6
			> 0x5985
-> 0x589f 22687 ('0x5985', 0)
ENTRIES REMAINING 13508
ENTRIES REMAINING 13507
0x589f:	mov	eax, 0 	5
0x58a4:	call	0xf0af 	5
61615 20192 44973

...

0x2488a:	add	rsp, 0x28 	4
0x2488e:	mov	eax, ebp 	2
0x24890:	pop	rbx 	1
0x24891:	pop	rbp 	1
0x24892:	pop	r12 	2
0x24894:	pop	r13 	2
0x24896:	pop	r14 	2
0x24898:	pop	r15 	2
0x2489a:	ret	 	1
ENTRIES REMAINING 472
ENTRIES REMAINING 471
ENTRIES REMAINING 470
ENTRIES REMAINING 469
ENTRIES REMAINING 468
ENTRIES REMAINING 467
ENTRIES REMAINING 466
ENTRIES REMAINING 465
ENTRIES REMAINING 464
ENTRIES REMAINING 463
ENTRIES REMAINING 462
ENTRIES REMAINING 461
ENTRIES REMAINING 460
0x24909:	cmp	al, 0x69 	2
0x2490b:	je	0x24bc0 	6
			> 0x24bc0
-> 0x24911 149777 ('0x24bc0', 0)
ENTRIES REMAINING 461
ENTRIES REMAINING 460
0x2491d:	lea	rcx, [rip + 0x4d7c] 	7
0x24924:	movzx	edx, dl 	3
0x24927:	movsxd	rax, dword ptr [rcx + rdx*4] 	4
0x2492b:	add	rax, rcx 	3
0x2492e:	jmp	rax 	3
rax
		->->->                                  indirect JMP
ENTRIES REMAINING 459
ENTRIES REMAINING 458
ENTRIES REMAINING 457
ENTRIES REMAINING 456
ENTRIES REMAINING 455
0x2496e:	lea	rdi, [rip + 0x4e03] 	7
0x24975:	movzx	edx, dl 	3
0x

0x25922:	jmp	0x259a2 	2
0x259a2
ENTRIES REMAINING 239
ENTRIES REMAINING 238
ENTRIES REMAINING 237
ENTRIES REMAINING 236
ENTRIES REMAINING 235
ENTRIES REMAINING 234
ENTRIES REMAINING 233
0x25949:	lea	rdx, [r11 + r14] 	4
0x2594d:	add	rsi, rax 	3
0x25950:	jmp	0x25965 	2
0x25965
ENTRIES REMAINING 233
ENTRIES REMAINING 232
ENTRIES REMAINING 231
ENTRIES REMAINING 230
0x2596f:	movabs	rdi, 0x20000002b 	10
0x25979:	bt	rdi, rax 	4
0x2597d:	jae	0x25958 	2
			> 0x25958
-> 0x2597f 153983 ('0x25958', 0)
ENTRIES REMAINING 231
ENTRIES REMAINING 230
ENTRIES REMAINING 229
ENTRIES REMAINING 228
ENTRIES REMAINING 227
ENTRIES REMAINING 226
ENTRIES REMAINING 225
ENTRIES REMAINING 224
ENTRIES REMAINING 223
0x25a25:	mov	rdi, r14 	3
0x25a28:	mov	qword ptr [rsp + 0x18], rdx 	5
0x25a2d:	call	0x24670 	5
149104 20192 449736
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

0x1028d:	cmp	eax, 0x73 	3
0x10290:	jg	0x104ce 	6
			> 0x104ce
-> 0x10296 66198 ('0x104ce', 0)
ENTRIES REMAINING 201
ENTRIES REMAINING 200
0x10719:	jmp	0x10755 	2
0x10755
ENTRIES REMAINING 200
ENTRIES REMAINING 199
0x146da:	cmp	eax, 4 	3
0x146dd:	je	0x147f3 	6
			> 0x147f3
-> 0x146e3 83683 ('0x147f3', 0)
ENTRIES REMAINING 200
ENTRIES REMAINING 199
0x15237:	cmp	eax, 0x61 	3
0x1523a:	je	0x15248 	2
			> 0x15248
-> 0x1523c 86588 ('0x15248', 0)
ENTRIES REMAINING 200
ENTRIES REMAINING 199
0x166a9:	cmp	dword ptr [rbp - 0x170], 3 	7
0x166b0:	je	0x166db 	2
			> 0x166db
-> 0x166b2 91826 ('0x166db', 0)
ENTRIES REMAINING 200
ENTRIES REMAINING 199
ENTRIES REMAINING 198
ENTRIES REMAINING 197
ENTRIES REMAINING 196
ENTRIES REMAINING 195
0x16b42:	jmp	0x16e18 	5
0x16e18
ENTRIES REMAINING 195
ENTRIES REMAINING 194
ENTRIES REMAINING 193
ENTRIES REMAINING 192
ENTRIES REMAINING 191
ENTRIES REMAINING 190
ENTRIES REMAINING 189
ENTRIES REMAINING 188
ENTRIES REMAINING 187
ENTRIES REMAINING 186
ENTRIES REMAINING 

0x23989:	fstp	st(0) 	2
0x2398b:	jmp	0x2398f 	2
0x2398f
ENTRIES REMAINING 63
ENTRIES REMAINING 62
ENTRIES REMAINING 61
ENTRIES REMAINING 60
0x23d6e:	fstp	st(0) 	2
0x23d70:	jmp	0x23d74 	2
0x23d74
ENTRIES REMAINING 60
ENTRIES REMAINING 59
ENTRIES REMAINING 58
ENTRIES REMAINING 57
0x23f4f:	movzx	edx, byte ptr [r15] 	4
0x23f53:	mov	byte ptr [r8], dl 	3
0x23f56:	test	al, 2 	2
0x23f58:	jne	0x23f6e 	2
			> 0x23f6e
-> 0x23f5a 147290 ('0x23f6e', 0)
ENTRIES REMAINING 58
ENTRIES REMAINING 57
0x2403f:	movzx	edx, byte ptr [r15] 	4
0x24043:	mov	byte ptr [r8], dl 	3
0x24046:	test	al, 2 	2
0x24048:	je	0x23eb9 	6
			> 0x23eb9
-> 0x2404e 147534 ('0x23eb9', 0)
ENTRIES REMAINING 58
ENTRIES REMAINING 57
ENTRIES REMAINING 56
ENTRIES REMAINING 55
0x2511c:	lea	rcx, [rip + 0x47bd] 	7
0x25123:	movzx	eax, dl 	3
0x25126:	movsxd	rax, dword ptr [rcx + rax*4] 	4
0x2512a:	add	rax, rcx 	3
0x2512d:	jmp	rax 	3
rax
		->->->                                  indirect JMP
ENTRIES REMAINING 54
ENTRIES REMAINING 53
0x25218:	no

0x1bc20:	jmp	0x1bc23 	2
0x1bc23
ENTRIES REMAINING 13
ENTRIES REMAINING 12
ENTRIES REMAINING 11
ENTRIES REMAINING 10
ENTRIES REMAINING 9
ENTRIES REMAINING 8
0x21654:	jmp	0x216a6 	2
0x216a6
ENTRIES REMAINING 8
ENTRIES REMAINING 7
ENTRIES REMAINING 6
ENTRIES REMAINING 5
ENTRIES REMAINING 4
ENTRIES REMAINING 3
ENTRIES REMAINING 2
ENTRIES REMAINING 1


In [7]:
int_list_to_hex_list(lineprogram_addresses[-200:])

['0x25ccd',
 '0x25ccd',
 '0x25cd1',
 '0x25cd1',
 '0x25cd1',
 '0x25cd1',
 '0x25cd6',
 '0x25cde',
 '0x25cf0',
 '0x25cf3',
 '0x25cf3',
 '0x25cf3',
 '0x25cf3',
 '0x25cf3',
 '0x25cf3',
 '0x25cf3',
 '0x25cf8',
 '0x25cf8',
 '0x25cf8',
 '0x25cf8',
 '0x25cf8',
 '0x25cff',
 '0x25cff',
 '0x25d07',
 '0x25d0a',
 '0x25d0c',
 '0x25d0e',
 '0x25d10',
 '0x25d20',
 '0x25d24',
 '0x25d24',
 '0x25d2a',
 '0x25d2c',
 '0x25d33',
 '0x25d33',
 '0x25d33',
 '0x25d33',
 '0x25d35',
 '0x25d50',
 '0x25d53',
 '0x25d57',
 '0x25d5c',
 '0x25d5c',
 '0x25d61',
 '0x25d61',
 '0x25d66',
 '0x25d72',
 '0x25d77',
 '0x25d77',
 '0x25d7e',
 '0x25d89',
 '0x25d89',
 '0x25d89',
 '0x25d95',
 '0x25d9d',
 '0x25d9d',
 '0x25da4',
 '0x25da4',
 '0x25dae',
 '0x25db2',
 '0x25dc0',
 '0x25dc0',
 '0x25dc4',
 '0x25dc4',
 '0x25dd7',
 '0x25de0',
 '0x25de0',
 '0x25de4',
 '0x25de4',
 '0x25de7',
 '0x25de7',
 '0x25de7',
 '0x25df7',
 '0x25dfc',
 '0x25e00',
 '0x25e00',
 '0x25e08',
 '0x25e0b',
 '0x25e0f',
 '0x25e12',
 '0x25e22',
 '0x25e22',
 '0x25e22',
 '0x

In [8]:
found = 0
nnotfound = 0
for ci in call_addresses:
    if ci in lineprogram_addresses:
        found+=1
    else:
        nnotfound+=1
print(found, nnotfound)
    

1916 0


In [9]:
print(obj.entry)
print(min(lineprogram_addresses), max(lineprogram_addresses))
min(visited_addresses), max(visited_addresses)

4214512
20192 155678


(20192, 155692)

In [10]:
print(textSectionStart , textSectionEnd)

20192 449736


In [11]:
20208 64 20144

SyntaxError: invalid syntax (364304702.py, line 1)

In [ ]:
if 0x70eb in lineprogram_addresses:
    print('yes')

In [ ]:
int_list_to_hex_list([i  for i in lineprogram_addresses if i >=0x10cf9-400 and i<=0x10cf9+400])